# Setting up for training

Defining imports and paths for images, labels and saved models

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from PIL import Image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K
from tensorflow.keras import losses
from tensorflow.keras import metrics
from sklearn.model_selection import train_test_split
# os.environ['MY_ENV'] = 'venv-metal'
# print(os.getenv("VIRTUAL_ENV"))

# all local paths, with these modified code runs elsewhere too
image_path = #'/directory'
label_path = #'/birch_labels.csv'
saved_model_path = #'/model_saving_directory/'

# image dimensions
img_dim = 224 * 1
applied_mode = 'grayscale' # 'grayscale' / 'rgb'
dim_3 = 1 # 1 or 3

Tensorflow settings changed for quicker training if desired or needed

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')

if len(physical_devices) > 0:
    # Enable memory growth for GPUs to prevent memory allocation issues
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

Define own loss functions if desired for better CNN training

In [ ]:
# Define the custom loss function
def mean_4th_power_loss(y_true, y_pred):
    # Calculate the error
    error = y_true - y_pred
    # Raise the error to the 4th power
    error_4th_power = K.pow(error, 4)
    # Return the mean of the 4th power of the error
    return K.mean(error_4th_power)

Prepare `image_paths` either from the label file's column filenames (label.csv has columns ['filename', feature vector]) or from the image directory

In [ ]:
def get_image_filenames(directory):
    image_files = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg')):
            image_files.append(filename)
    return image_files

In [ ]:
df = pd.read_csv(label_path)

image_files = get_image_filenames(image_path)
print(image_files)
image_paths = [image_path + '/' + i for i in df['filename']]

* Preprocessing images, inputs 3024 x 4032, 4480 x 6720
* Split into training and validation data sets, choose `target_feature` and `target_type` for CNN training
* add three rotated images to the training image set, increase label vector sizes accordingly

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Function to load and preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_dim, img_dim, dim_3), color_mode = applied_mode)  # Resize to 224x224 (for example)
    # img = load_img(image_path, target_size=(224, 224))  # Resize to 224x224 (for example)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

# Apply preprocessing to training and validation data
all_images = []

failed_images = []  # List to store failed image paths

# Helper function to process each image with error handling
def process_image_with_error_handling(img_path):
    try:
        return preprocess_image(img_path)  # Attempt to process image
    except Exception as e:
        failed_images.append(img_path)  # If an error occurs, store the image path
        return None  # Return None or another indicator for failure

# Use the helper function in the list comprehension
approved_images = [process_image_with_error_handling(img_path) for img_path in image_paths]

# Optionally, filter out failed images (if you don't want `None` values in your list)
approved_images = [img for img in approved_images if img is not None]

# Now, `failed_images` will contain the paths of the images that caused an error.
print(f"Failed images: {failed_images}")
approved_image_paths = [path for path in image_paths if path not in failed_images]

In [ ]:
features = ['weeping', 'antigravitropic', 'main_trunks', 'canopy_breadth', 'primary_branches', 'branch_density', 'orientation']
target_feature = 'orientation'
target_type = 'binary' # binary, multi-class, regression

# Remove failed images from train labels
df = df[~df['filename'].isin([i.split("/")[-1] for i in failed_images])]
labels = df[target_feature].values
n_classes = len(df[target_feature].unique())
# paths after taking invalid out
applied_image_paths = approved_image_paths

train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(applied_image_paths, labels, test_size=0.3, random_state=42)
train_images = [approved_images[approved_image_paths.index(i)] for i in train_image_paths]
val_images = [approved_images[approved_image_paths.index(i)] for i in val_image_paths]

In [ ]:
# add four-way rotations for pics if needed

# Define a function to perform the transformations
def augment_image(image):
    # Perform a left rotation (90 degrees counterclockwise)
    left_rot = np.rot90(image, k=1, axes=(0, 1))  # Rotating counterclockwise by 90 degrees
    
    # Perform a right rotation (90 degrees clockwise)
    right_rot = np.rot90(image, k=3, axes=(0, 1))  # Rotating clockwise by 90 degrees
    
    # Perform a vertical flip (flip up and down)
    vert_flip = np.flipud(image)  # Flipping vertically
    
    # Return the original image and the transformed ones
    return np.array([image, left_rot, right_rot, vert_flip])

# Apply augmentations to all images in the dataset
augmented_images = []

for img in train_images:
    augmented_images.append(augment_image(img))

# Convert the list into a numpy array
augmented_images = np.concatenate(augmented_images, axis=0)

print(augmented_images.shape)
train_images = augmented_images
train_labels = [item for item in train_labels for _ in range(4)]
train_labels = np.array(train_labels).reshape(len(train_labels), 1)

# Creating and training the model

Create a Convolutional Neural Network 

In [ ]:
from tensorflow.keras import layers, models

def create_model_bin(input_shape=(img_dim, img_dim, dim_3)):
    model = models.Sequential()
    
    # Add convolutional layers
    model.add(layers.Conv2D(8, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(8, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # For binary classification (use 'softmax' for multi-class)
    
    model.compile(optimizer=Adam(learning_rate=0.01), loss=losses.BinaryCrossentropy(), metrics=['accuracy'])
    
    return model

def create_model_multi(input_shape=(img_dim, img_dim, dim_3)):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(4, (5, 5), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(4, (5, 5), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(8, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    # model.add(layers.MaxPooling2D((2, 2)))

    # Flatten and dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))

    # Output layer for ordinal regression (single unit with linear activation)
    model.add(layers.Dense(1, activation='linear'))  # Output a single continuous value
    
    # Compile the model with MSE loss for ordinal regression
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model
    

def create_model_reg(input_shape=(img_dim, img_dim, dim_3)):
    model = models.Sequential()
    
    # Add convolutional layers
    model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(4, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(4, activation='relu'))
    
    # Output layer for regression with linear activation
    model.add(layers.Dense(1, activation='linear'))  # For predicting a continuous value
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return model

# Create the model
if target_type == "binary":
    model = create_model_bin()
elif target_type == "multi-class":
    model = create_model_multi()
else:
    model = create_model_reg()

Train the model

In [ ]:
# Convert lists of images to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(train_images, train_labels, 32)
test_gen = DataGenerator(val_images, val_labels, 32)

# Train the model
history = model.fit(train_gen, 
                    epochs=10, 
                    validation_data=test_gen)

Evaluate the model

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_images, val_labels)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

In [ ]:
# run predictions on chosen set of images
predictions = model.predict(val_images)

# plot training
pd.DataFrame(history.history).plot()
plt.title('Train and validation by epoch')
plt.xlabel('epoch')
plt.ylabel(target_feature + ' error')
# plt.savefig(saved_model_path + 'temp')
plt.show()

In [ ]:
plt.scatter(predictions, val_labels)
plt.title('Prediction vs actual in validation')
plt.xlabel('predicted value')
plt.ylabel('actual value')
# plt.savefig(saved_model_path + 'temp')
plt.show()

# Save and load model

Save model

In [ ]:
model.save(saved_model_path + 'model_orientation.keras')  # Saves the model in the SavedModel format

Load model

In [ ]:
loaded_model = tf.keras.models.load_model(saved_model_path + 'model_antigravitropic.keras')